In [323]:
import pandas as pd
import numpy as np
import pytz
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [324]:
csv_file_path = '/home/lucas/UFOP/ple_2020/analise_midias_sociais/final-work/data/all_pp_posts_bolsonaro_posts.csv'
df_candidate = pd.read_csv(csv_file_path,encoding = 'utf-8')

In [325]:
# convertendo o tipo data que está como string em objeto no formato data
df_candidate["created_time"] = pd.to_datetime(df_candidate['created_time'])

In [326]:
df_candidate.head(15)

,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-03-22 18:47:55+00:00,211857482296579_1826486634166981,agor videoconferenc president jair bolsonar pr...,- agora por videoconferencia o presidente jair...,23151,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,129465,147,318,21,112,10345,True
1,2020-03-22 16:30:39+00:00,211857482296579_1826339070848404,inscrico seleca profission atuaca program medi...,- as inscricoes para a selecao de 5.800 profis...,5703,mobile_status_update,None,52238,128,61,5,43,2514,True
2,2020-03-22 13:37:26+00:00,211857482296579_1826130007535977,ministeri saud ministr mandett reunia virtual ...,- estamos no ministerio da saude com o ministr...,11818,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,97986,73,141,16,65,6292,True
3,2020-03-22 11:07:01+00:00,211857482296579_1825937284221916,govern federal distribu milho test rap covid t...,- o governo federal distribui milhoes de teste...,39284,mobile_status_update,None,165539,88,393,20,68,16747,True
4,2020-03-22 02:06:48+00:00,211857482296579_1825422034273441,cham precauca medic barat cas venh ser comprov...,- isso se chama precaucao . o medicamento e ba...,27236,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9052732...,162513,968,318,88,509,20723,True
5,2020-03-21 22:22:48+00:00,211857482296579_1825061764309468,reconhec seriedad moment temor muit brasileir ...,- reconheco a seriedade do momento e o temor d...,40794,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9065012...,305358,918,478,266,1267,48834,True
6,2020-03-21 18:36:01+00:00,211857482296579_1824728371009474,hospital albert einstein possivel cur pacient ...,- hospital albert einstein e a possivel cura d...,201803,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/75...,215237,480,815,60,431,34278,True
7,2020-03-21 11:33:30+00:00,211857482296579_1824261207722857,dois ded pros ratinh assist link youtub,- dois dedos de prosa com ratinho . - assista ...,33971,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/83...,106117,336,317,113,624,15368,True
8,2020-03-21 02:37:00+00:00,211857482296579_1823872341095077,assin med provisor deix clar ser competenc fed...,- assinei medida provisoria deixando claro ser...,9174,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,112873,181,200,106,1121,7582,True
9,2020-03-21 00:16:41+00:00,211857482296579_1823743497774628,via eduard bolsonar,via eduardo bolsonaro,8150,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p600x60...,79762,435,111,13,160,6480,True


In [302]:
n_total_posts = len(df_candidate)
print('número total de posts: {0}'.format(str(n_total_posts)))

número total de posts: 3646


In [303]:
n_posts_with_textual_messages = df_candidate[df_candidate.has_textual_message == True].shape[0]
n_posts_without_textual_messages = df_candidate[df_candidate.has_textual_message == False].shape[0]

print('número de posts com mensagens textuais: {0}'.format(str(n_posts_with_textual_messages)))
print('número de posts sem mensagens textuais: {0}'.format(str(n_posts_without_textual_messages)))

número de posts com mensagens textuais: 3394
número de posts sem mensagens textuais: 252


In [180]:
# Devemos remover esses posts sem menssagem antes de continuar a nossa análise?
# renomear depois o df caso sim

#filter_mask  = df_candidate['has_textual_message'] == True
#df_candidate = df_candidate[filter_mask]
#print('número de posts do df desconsiderando os posts sem menssagens textuais: {0}'.format(str(len(df_candidate))))

número de posts do df desconsiderando os posts sem menssagens textuais: 3394


In [304]:
# separando o df de treinamento (30%) e o df de testes(70%) de maneira 'embaralhada'
df_training, df_test = train_test_split(df_candidate,train_size=0.7,test_size=0.3,shuffle=True)

In [305]:
print('número de posts do conjunto de treinamento (70%): {0}'.format(str(len(df_training))))
df_training.head(5)

número de posts do conjunto de treinamento (70%): 2552


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
3404,2014-04-25T21:06:28+0000,211857482296579_313017942180532,bolsonar candidat president republ deput feder...,bolsonaro candidato a presidente da republica ...,26847,added_photos,https://scontent.xx.fbcdn.net/v/t31.0-8/s720x7...,45582,2,1,1,0,12,True
2523,2016-07-25T21:59:59+0000,211857482296579_663456960469960,hoj centr rio janeir fascist golpist nao passa...,hoje no centro do rio de janeiro . `` fascista...,18331,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,59167,4282,108,27,91,1997,True
395,2020-01-22T19:15:44+0000,211857482296579_486119865653370,,,3119,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/72...,27177,69,75,24,65,4151,False
1057,2019-01-25T17:23:23+0000,211857482296579_1356941737788142,,,2290,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/p720x72...,44110,116,581,16,24,4829,True
1736,2017-12-07T18:48:59+0000,211857482296579_964541977028122,bolsonar anapol go,bolsonaro em anapolis go,8432,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/p5...,9041,86,45,6,30,792,True


In [306]:
print('número de posts do conjunto de teste (30%): {0}'.format(str(len(df_test))))
df_test.head(5)

número de posts do conjunto de teste (30%): 1094


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
3608,2013-08-27T14:24:13+0000,211857482296579_234599850022342,jornal dia govern pt nao cans ment omit sobr v...,jornal o dia : governo do pt nao se cansa de m...,194,shared_story,https://external.xx.fbcdn.net/safe_image.php?d...,239,0,0,0,0,0,True
3159,2015-02-19T17:55:51+0000,211857482296579_446821765466815,dezembr entrevist tv boas nov program lev ar j...,em dezembro fui entrevistado pela tv boas nova...,1253,shared_story,https://external.xx.fbcdn.net/safe_image.php?d...,4603,0,0,0,0,0,True
880,2019-06-25T17:16:54+0000,211857482296579_1499958480153133,receb nest tard palaci planalt visit grand ami...,- recebi nesta tarde no palacio do planalto a ...,8767,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/65...,66461,150,379,20,76,4378,True
2400,2016-10-22T10:42:37+0000,211857482296579_708745785941077,bolsonar cheg sao paul congonh,bolsonaro chega em sao paulo ( congonhas ) .,11469,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/14...,57269,850,428,63,238,8402,True
584,2019-11-12T00:42:31+0000,211857482296579_1662477187234594,junt ministeri desenvolv regional entreg milh ...,- junto com o ministerio do desenvolvimento re...,24267,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/71...,65857,108,697,9,22,6959,True


In [272]:
#============================================================================================
# realizando o tf idf no df['pre_processed_message'] de treinamento
#============================================================================================

# instanciando o objeto para fazer o tf-idf
tfidf_vectorizer = TfidfVectorizer(analyzer='word')

# converte os documentos em matrizes
tfidf_matrix = tfidf_vectorizer.fit_transform(df_training['pre_processed_message'])

# obtendo as palavras (tokens)
tfidf_tokens = tfidf_vectorizer.get_feature_names()

# gerando o tf-idf em um df
# colunas são as palavras processadas
# linhas são os documentos (post 1, post 2,post 3,post 4)
df_tfidf_df = pd.DataFrame(data = tfidf_matrix.toarray(),columns = tfidf_tokens)


print("\nTF-IDF\n")
df_tfidf_df.head(10)


TF-IDF



,abaix,abandon,abastec,abat,abav,abcz,abdelmassih,abenco,aberraco,abert,...,zarattin,ze,zec,zelos,zer,zl,zoeir,zon,zueir,zuer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [307]:
# Aplicando uma máscara para identificar o tf-idf de certas palavras

filter_mask  = df_tfidfvect['zarattin'] != 0.0
df_tfidf_token_mask = df_tfidfvect[filter_mask]

print(df_tfidf_token_mask)

      abaix  abandon  abastec  abat  abav  abcz  abdelmassih  abenco  \
463     0.0      0.0      0.0   0.0   0.0   0.0          0.0     0.0   
1342    0.0      0.0      0.0   0.0   0.0   0.0          0.0     0.0   

      aberraco  abert  ...  zarattin   ze  zec  zelos  zer   zl  zoeir  zon  \
463        0.0    0.0  ...  0.430778  0.0  0.0    0.0  0.0  0.0    0.0  0.0   
1342       0.0    0.0  ...  0.356042  0.0  0.0    0.0  0.0  0.0    0.0  0.0   

      zueir  zuer  
463     0.0   0.0  
1342    0.0   0.0  

[2 rows x 6535 columns]


In [ ]:
#============================================================================================
# Implementacao do lda
#============================================================================================



In [224]:
cv = CountVectorizer(max_df = 0.8, min_df = 2)
df = cv.fit_transform(df_candidate['pre_processed_message'])

In [20]:
model_lda = LatentDirichletAllocation(n_components = 10)
model_lda.fit(df)

LatentDirichletAllocation()

In [14]:
for index, topic in enumerate(model_lda.components_):
    print(f'Top 10 words for #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for #0
['brasil', 'milit', 'alun', 'federal', 'public', 'cas', 'ensin', 'program', 'educaca', 'escol']


Top 10 words for #1
['pod', 'pov', 'pt', 'vid', 'polit', 'tud', 'acim', 'tod', 'nao', 'brasil']


Top 10 words for #2
['cam', 'deput', 'glob', 'direit', 'sobr', 'jornal', 'brasil', 'verdad', 'bolsonar', 'nao']


Top 10 words for #3
['part', 'hoj', 'milit', 'tod', 'janeir', 'grand', 'abrac', 'rio', 'brasil', 'obrig']


Top 10 words for #4
['sao', 'seguranc', 'econom', 'nao', 'brasil', 'med', 'public', 'ministr', 'govern', 'ministeri']


Top 10 words for #5
['infraestrutur', 'saud', 'nov', 'ministr', 'milho', 'estad', 'econom', 'govern', 'brasil', 'ministeri']


Top 10 words for #6
['tod', 'ser', 'dilm', 'president', 'eduard', 'sao', 'jair', 'deput', 'nao', 'bolsonar']


Top 10 words for #7
['agor', 'flavi', 'sobr', 'eduard', 'brasil', 'tod', 'youtub', 'link', 'jair', 'bolsonar']


Top 10 words for #8
['jair', 'dilm', 'sobr', 'contr', 'deput', 'lei', 'brasil', 'nao', 'pt'

In [19]:
feature_names = cv.get_feature_names()
number_top_words = 10
for topic_idx, topic in enumerate(model_lda.components_):
    message = "Topic {0}: ".format(str(topic_idx))
    message += " ".join([feature_names[i]
                         for i in topic.argsort()[:-number_top_words - 1:-1]])
    print(message)
print()

Topic 0: brasil todos nao acima tudo governo deus sao pais pt
Topic 1: governo ministerio brasil programa bilhoes milhoes anos federal ano economia
Topic 2: sao pt nao paulo verdade sobre jornal comissao camara so
Topic 3: bolsonaro rio nao carlos janeiro brasil ministerio dilma obrigado flavio
Topic 4: bolsonaro jair youtube link brasil sobre eduardo entrevista obrigado presidente
Topic 5: bolsonaro federal nao eduardo lei brasil deputado sp canal sao
Topic 6: todos voto nao impresso bolsonaro hoje abraco brasil povo obrigado
Topic 7: nao bolsonaro educacao direitos militar humanos brasil sobre escolas rio
Topic 8: nao todos brasil presidente ser ministro bolsonaro boa apoio estado
Topic 9: obrigado nao pt contra brasil publica psol crime grande ministerio



In [334]:
#===========================================================================
# obter o dataframe com os posts do candidato no ano de 2018
#===========================================================================

time_mask = (df_candidate['created_time'] >= '2018-01-01') & (df_candidate['created_time'] <= '2018-12-31')

df_election = df_candidate[time_mask]

In [342]:
#===========================================================================
# ordenando df_election por reacoes sad 
#===========================================================================

election_df_sorted_by_sad = df_election.sort_values('reactions_sad', ascending=False) # ordenando o df por sad

sorted_df_by_sad

,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
1598,2018-02-11 15:08:32+00:00,211857482296579_1011047899044196,cad vaz pov liberdad pen margin viv encarcer s...,- cadeias vazias povo sem liberdade . - se tiv...,133052,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/27...,97723,242,678,7916,1838,3930,True
1167,2018-10-03 20:23:55+00:00,211857482296579_261038144548191,mae fal realment preocup tod barc deus permit ...,- uma mae fala o que realmente preocupa a todo...,40588,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/38...,64315,131,348,7726,232,2447,True
1675,2018-01-05 18:53:09+00:00,211857482296579_982212701927716,cris seguranc rio grand nort,- a crise da seguranca no rio grande do norte .,19851,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/24...,31478,41,173,7321,2332,128,True
1157,2018-10-08 00:28:34+00:00,211857482296579_250825412292718,bolsonar fal brasil segund turn link youtub,bolsonaro fala ao brasil . estamos no segundo ...,236620,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/42...,454568,3032,1291,6250,3337,218849,True
1149,2018-10-11 13:07:31+00:00,211857482296579_347058056038010,assist tir concluso,assista e tire suas conclusoes :,31554,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/41...,27288,368,408,5563,15463,174,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,2018-05-06 01:53:15+00:00,211857482296579_1062903607191958,aniversari corp fuzileir nav,- 210 aniversario do corpo de fuzileiros navais .,724,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/29...,7599,6,52,0,4,449,True
1562,2018-03-07 16:17:42+00:00,211857482296579_1025592000923119,constat viag asi importanc dao cienc tecnolog ...,- constatamos em nossa viagem a asia a importa...,5934,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/27...,9270,21,99,0,12,598,True
1574,2018-02-26 13:01:10+00:00,211857482296579_1020159428133043,minut dest segundafeir japa,- um minuto desta segunda-feira no japao .,2071,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/27...,13516,20,166,0,7,823,True
1607,2018-02-08 12:53:16+00:00,211857482296579_1009036279245358,show rural cascavelpr visit exposica parabens ...,- show rural em cascavel/pr . - visita a expos...,2578,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/27...,9337,12,82,0,5,564,True
